## 1. 라이브러리 설치

In [1]:
!pip install pdfplumber

## 2. Bedrock 설정 및 호출 함수 정의

In [2]:
import json
import boto3
from pprint import pprint

# Bedrock runtime 생성
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime"
)

In [3]:
def invoke_bedrock(
    query: str,
    model_id: str = "anthropic.claude-3-sonnet-20240229-v1:0",
    max_tokens: int = 4096,
    temperature: float = 1.0,
    top_p: float = 1.0,
    top_k: int = 250,
) -> str:

    # Request body 작성
    request_data = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": query},
                ],
            }
        ],
    }
    request_body = json.dumps(request_data)

    # 모델 호출
    response = bedrock_runtime.invoke_model(
        body=request_body,
        modelId=model_id,
        contentType="application/json",
        accept="application/json"
    )

    response_body = json.loads(response['body'].read())
    answer = response_body['content'][0]['text']
    return answer

In [4]:
result = invoke_bedrock(query="무역실무에 대한 퀴즈 한 개 만들고, 정답도 알려줘")
print(result)

네, 무역실무 관련 퀴즈 하나를 내겠습니다.

질문: 수출상품을 포장한 후에 발행되는 중요한 서류는 무엇일까요?
1) 선하증권(Bill of Lading)
2) 무역거래계약서(Trade Contract)
3) 포장명세서(Packing List)
4) 원산지증명서(Certificate of Origin)

정답: 3) 포장명세서(Packing List)입니다.

포장명세서는 수출상품을 포장한 후 작성되며, 포장 상자 수량, 중량, 부피 등의 내용물 정보를 기재합니다. 이는 선적과 통관 시 매우 중요한 무역서류로 활용됩니다.


## 3. PDF 문서 읽는 함수 정의¶

In [5]:
import pdfplumber

def get_texts_from_pdf(filepath: str, start_page:int, page_number:int):
    with pdfplumber.open(filepath) as pdf:
        number_of_pages = len(pdf.pages)
        end_page = start_page + page_number
        if end_page > number_of_pages:
            end_page = number_of_pages
            
        contents = ""
        for i in range(start_page, end_page):
            page = pdf.pages[i]
            contents += page.extract_text()
            contents += "\n\n"
        return contents

In [6]:
contents = get_texts_from_pdf(filepath="./rag_data/0412통합본-알기_쉬운_무역실무_길라잡이.pdf", start_page=0, page_number=1)
print(contents)

2023
개정판
# 알기쉬운
무역
실무
길 잡이
라




## 5. (Optional) 문제 만들기

In [7]:
content = get_texts_from_pdf(filepath="./rag_data/0412통합본-알기_쉬운_무역실무_길라잡이.pdf", start_page=5, page_number=4)
prompt = f"""

당신은 PDF 문서에서 무역 실무에 대한 퀴즈를 만드는 AI 입니다.
주어진 <content> 안의 문서 내용을 이해했는지 확인하기 위한 퀴즈 5개를 만드세요.
각 퀴즈는 4개의 보기중 정답 하나를 고르는 문제입니다.

<content>
{content}
</content>

정답은 맨 아래 생성한 퀴즈의 정답을 작성합니다.
"""
result = invoke_bedrock(query=prompt, temperature=0, top_p=0)
print(result)

1. 무역업 창업을 위해서는 먼저 업종을 선정한 후 사업계획을 수립해야 합니다. 다음 중 사업계획 수립 단계에서 검토해야 할 사항이 아닌 것은?

가. 시장 타당성 조사
나. 업종 선정
다. 법인 설립 등기
라. 사업 인·허가 취득

2. 창업사업계획 승인제도에 대한 설명으로 옳지 않은 것은?

가. 창업일로부터 7년이 경과하지 않은 중소기업자가 대상입니다.
나. 공장설립 관련 29개 법률에 의한 53개 인·허가 사항을 일괄 처리합니다.
다. 사업계획승인신청서 작성 및 제출을 대행해 주는 서비스가 있습니다.
라. 사업계획승인 신청 후 10일 이내에 승인 여부를 결정합니다.

3. 중소기업 창업지원법상 창업으로 인정되지 않는 경우는?

가. 타인의 기업을 승계하여 승계 전과 동일한 기업인 경우
나. 법인 전환 또는 기업 형태를 변경하여 변경 전의 사업과 동종의 사업인 경우
다. 폐업 후 사업을 개시하여 폐업 전의 사업과 동종의 사업을 계속하는 경우
라. 위 모든 경우가 창업으로 인정되지 않습니다.

4. 창업중소기업에 대한 세제 지원 중 옳지 않은 것은?

가. 창업 후 소득 발생 연도부터 4년간 매년 납부할 법인세의 50%를 감면합니다.
나. 창업벤처기업의 사업용 재산 취득세 및 등록세를 면제합니다.
다. 창업벤처기업의 사업용 재산에 대해 창업일로부터 5년간 재산세 및 종합토지세의 100%를 감면합니다.
라. 창업 후 2년 이내에 금융기관 융자 관련 문서에 대한 인지세를 면제합니다.

5. 중소기업 창업자금 지원과 관련된 기관이 아닌 것은?

가. 중소벤처기업진흥공단
나. 기술신용보증기금
다. 한국발명진흥회
라. 국민연금공단

정답:
1. 다
2. 라
3. 라 
4. 다
5. 라


## 6. 사용자 이해도 분석하기

In [8]:
quiz = result
my_answer = "1. 다, 2.라 3. 다. 4. 가 5. 가" #4번(세제지원)과 5번(창업자금 지원) 에 대한 답변이 틀림
my_answer_prompt = f"""

당신은 사용자에 대한 무역 퀴즈의 이해도를 판단하는 AI 입니다
<quiz> 태그 안에 AI가 생성한 문제와 정답이 있습니다.
그리고 사용자가 생각한 답변은 <my answer> 태그 안에 있습니다.

<quiz>
{quiz}
</quiz>

<my answer>
{my_answer}
</my answer>

사용자의 답변과 실제 답변을 비교해서 사용자가 잘 이해하고 있는 부분과 잘 이해하고 있지 않은 부분을 나눠서 설명해주세요.

"""
user_analysis = invoke_bedrock(query=my_answer_prompt, temperature=0, top_p=0)
print(user_analysis)

사용자님의 답변을 검토해보겠습니다.

잘 이해하고 있는 부분:
1. 법인 설립 등기는 사업계획 수립 단계가 아닌 후속 절차임을 정확히 알고 계십니다.
2. 창업사업계획 승인 후 10일 이내에 승인 여부를 결정한다는 사실을 잘 이해하고 계십니다.
3. 타인 기업 승계, 법인 전환, 폐업 후 재개업 등의 경우 창업으로 인정되지 않는다는 점을 정확히 알고 계십니다.

잘 이해하고 있지 않은 부분:
4. 창업벤처기업의 사업용 재산에 대한 재산세 및 종합토지세 감면 기간이 5년이 아니라 창업일로부터 4년간입니다.
5. 중소벤처기업진흥공단과 기술신용보증기금은 중소기업 창업자금 지원 기관이지만, 한국발명진흥회와 국민연금공단은 창업자금 지원과 관련이 없습니다.

전반적으로 무역 창업 관련 기본 지식을 잘 갖추고 계시지만, 일부 세부 사항에 대해서는 보완이 필요해 보입니다. 창업 시 받을 수 있는 세제 혜택과 지원 기관에 대해 다시 한번 확인하시면 좋을 것 같습니다.


## 7. 사용자에 맞는 문제 내기

In [9]:
content = get_texts_from_pdf(filepath="./rag_data/0412통합본-알기_쉬운_무역실무_길라잡이.pdf", start_page=5, page_number=4)
new_quiz_prompt = f"""

당신은 사용자의 무역 실무에 대한 이해도를 기반으로 PDF 에서 새로운 퀴즈를 만드는 AI 입니다.
<quiz> 태그 안에 AI가 생성한 문제와 정답이 있습니다.
그리고 각 문제에 대한 사용자의 이해도는 <user analysis> 태그 안에 있습니다.

<quiz>
{quiz}
</quiz>

<user analysis>
{user_analysis}
</user analysis>

위에 있는 퀴즈와 사용자의 이해도에 따른 정보를 참고해서
주어진 <content> 안의 문서에서 사용자가 틀린 문제에 대해서 이해도를 높히기 위한 퀴즈 3개를 만드세요.

<content>
{content}
</content>

정답은 맨 아래 생성한 퀴즈의 정답을 작성합니다.

"""
new_quiz = invoke_bedrock(query=new_quiz_prompt, temperature=0, top_p=0)
print(new_quiz)

사용자님의 이해도를 높이기 위한 퀴즈를 만들겠습니다.

<quiz>
1. 창업벤처기업의 사업용 재산에 대한 재산세 및 종합토지세 감면 기간은 얼마인가?

가. 창업일로부터 3년간
나. 창업일로부터 4년간
다. 창업일로부터 5년간
라. 창업일로부터 7년간

2. 다음 중 중소기업 창업자금 지원 기관이 아닌 곳은?

가. 중소벤처기업진흥공단
나. 기술신용보증기금  
다. 국민연금공단
라. 소상공인지원센터

3. 창업사업계획 승인제도에서 창업자의 사업계획승인신청서 작성 및 제출을 대행해주는 기관은 무엇인가?

가. 중소기업상담회사
나. 중소벤처기업부
다. 지방자치단체
라. 한국발명진흥회

</quiz>

정답:
1. 나
2. 다
3. 가
